### Data Model
1. Data Export : CSV -> JSON
2. Data Import : JSON -> CSV 
3. Create test train spit
4. Write TFRecord : JSON -> TFRecord
5. Data import for Schneider
6. Check data

##### Data Exports

In [2]:
#import pandas as pd
import csv, os, sys
import operator
import tensorflow as tf
import json
sys.path.append('/home/rai00007/Desktop/zooniverse/Data/')
from utils import dataset_util

In [3]:
Project_filepath = "/home/rai00007/Desktop/zooniverse/Data/images/"# "/data/lucifer1.2/users/rai00007/"

In [4]:
import glob
lst = (glob.glob("/home/rai00007/Desktop/zooniverse/Data/images/*.jpg"))
lst2 = []
for l in lst:
    lst2.append(l[-14:]) #list of all images that schnider used

In [5]:
gold_standard_bb = {}
with open('/home/rai00007/Desktop/zooniverse/Data/GoldStandardBoundBoxCoord.csv', 'r') as csvfile:
    csvdata = csv.reader(csvfile, delimiter=',')
    header = next(csvdata)
    col_mapper = {k: i for i, k in enumerate(header)}
    for i, row in enumerate(csvdata):
        filename = row[col_mapper['filename']]
        if filename in lst2:
            if filename not in gold_standard_bb.keys():
                gold_standard_bb[filename] = []
            gold_standard_bb[filename].append({k: row[v] for k, v in col_mapper.items() })
            if i%5000 == 0:
                print("performing iteration : %s" % i)
    #print gold_standard_bb

performing iteration : 0
performing iteration : 5000
performing iteration : 10000
performing iteration : 15000


In [6]:
all_images = {}
with open('/home/rai00007/Desktop/zooniverse/Data/all_images.csv', 'r') as csvfile:
    csvdata = csv.reader(csvfile, delimiter=',')
    header = next(csvdata)
    col_mapper = {k: i for i, k in enumerate(header)}
    for i, row in enumerate(csvdata):
        filename = row[col_mapper['CaptureEventID']]
        if filename not in all_images.keys():
            all_images[filename] = []
        all_images[filename].append({k: row[v] for k, v in col_mapper.items() })
        if i%50000 == 0:
            print("performing iteration : %s" % i)
        #print ', '.join(row)
    #print all_images

performing iteration : 0
performing iteration : 50000
performing iteration : 100000
performing iteration : 150000
performing iteration : 200000
performing iteration : 250000
performing iteration : 300000
performing iteration : 350000
performing iteration : 400000
performing iteration : 450000
performing iteration : 500000
performing iteration : 550000
performing iteration : 600000
performing iteration : 650000
performing iteration : 700000
performing iteration : 750000
performing iteration : 800000
performing iteration : 850000
performing iteration : 900000
performing iteration : 950000
performing iteration : 1000000
performing iteration : 1050000
performing iteration : 1100000
performing iteration : 1150000
performing iteration : 1200000
performing iteration : 1250000
performing iteration : 1300000
performing iteration : 1350000
performing iteration : 1400000
performing iteration : 1450000
performing iteration : 1500000
performing iteration : 1550000
performing iteration : 1600000
per

In [7]:
consensus_data = {}
with open(os.path.join(Project_filepath, '/home/rai00007/Desktop/zooniverse/Data/consensus_data.csv'), 'r') as csvfile:
    csvdata = csv.reader(csvfile, delimiter=',')
    header = next(csvdata)
    col_mapper = {k: i for i, k in enumerate(header)}
    for i, row in enumerate(csvdata):
        filename = row[col_mapper['CaptureEventID']]
        if filename not in consensus_data.keys():
            consensus_data[filename] = []
        consensus_data[filename].append({k: row[v] for k, v in col_mapper.items() })
        if i%50000 == 0:
            print("performing iteration : %s"% i)
        #print ', '.join(row)
    #print consensus_data

performing iteration : 0
performing iteration : 50000
performing iteration : 100000
performing iteration : 150000
performing iteration : 200000
performing iteration : 250000
performing iteration : 300000


CaptureEventID - Consensus Data <br>
Metadata {SiteID, Location, Datetime} - Consensus Data <br>
Images [{ path} - all_images, dimension - GoldStandardBoundBoxCoord] <br>
Observation [bounding box] - GoldStandardBoundBoxCoord

##### Helper functions

In [8]:
"""This function reads a raw image, resizes it with aspect ratio preservation and returns the byte string"""
from PIL import Image
import numpy as np
import io



def resize_jpeg(image,  max_side):
    """ Take Raw JPEG resize with aspect ratio preservation
         and return bytes
    """
    img = Image.open(image)
    img.thumbnail([max_side, max_side], Image.ANTIALIAS)
    b = io.BytesIO()
    img.save(b, 'JPEG')
    image_bytes = b.getvalue()
    return image_bytes

In [9]:
def class_text_to_int(row_label):

    if row_label == 'Aardwolf': return 1
    elif row_label == 'Aardvark': return 2
    elif row_label == 'Baboon': return 3
    elif row_label == 'Buffalo': return 4
    elif row_label == 'BuffaloYouth': return 5
    elif row_label == 'BuffCrestedBustard': return 6
    elif row_label == 'Bushbuck': return 7
    elif row_label == 'CattleEgret': return 8 #CatteEgret
    elif row_label == 'Cheetah': return 9
    elif row_label == 'DikDik': return 10
    elif row_label == 'DistantAnimal': return 11
    elif row_label == 'Eland': return 12
    elif row_label == 'Elephant': return 13
    elif row_label == 'ElephantYouth': return 14
    elif row_label == 'GazelleGrants': return 15
    elif row_label == 'GazelleGrantsFemale': return 16
    elif row_label == 'GazelleGrantsMale': return 17
    elif row_label == 'GazelleThomsons': return 18
    elif row_label == 'GazelleThomsonsFemale': return 19
    elif row_label == 'GazelleThomsonsMale': return 20
    elif row_label == 'GazelleThomsonsYouth': return 21
    elif row_label == 'Giraffe': return 22
    elif row_label == 'GreyBackedFiscal': return 23
    elif row_label == 'GuineaFowl': return 24
    elif row_label == 'GuineaFowlYouth': return 25
    elif row_label == 'Hare': return 26
    elif row_label == 'Hartebeest': return 27
    elif row_label == 'Hippopotamus': return 28
    elif row_label == 'HotAirBalloon': return 29
    elif row_label == 'Human': return 30
    elif row_label == 'HyenaSpotted': return 31
    elif row_label == 'Impala': return 32
    elif row_label == 'ImpalaFemale': return 33
    elif row_label == 'ImpalaMale': return 34
    elif row_label == 'ImpalaYouth': return 35
    elif row_label == 'Jackal': return 36
    elif row_label == 'KoriBustard': return 37
    elif row_label == 'Leopard': return 38
    elif row_label == 'LionFemale': return 39 
    elif row_label == 'LionMale': return 40
    elif row_label == 'Mongoose': return 41
    elif row_label == 'Ostrich': return 42
    elif row_label == 'OtherBird': return 43
    elif row_label == 'Oxpecker': return 44
    elif row_label == 'Porcupine': return 45
    elif row_label == 'Reedbuck': return 46
    elif row_label == 'ReedbuckMale': return 47
    elif row_label == 'ReedbuckFemale': return 48
    elif row_label == 'ReedbuckYouth': return 49
    elif row_label == 'Rhinoceros': return 50
    elif row_label == 'Rodent': return 51
    elif row_label == 'SecretaryBird': return 52
    elif row_label == 'Serval': return 53
    elif row_label == 'SpottedHyena': return 54
    elif row_label == 'Stork': return 55
    elif row_label == 'SuperbStarling': return 56
    elif row_label == 'Topi': return 57
    elif row_label == 'TopiMale': return 58
    elif row_label == 'TopiFemale': return 59
    elif row_label == 'Truck': return 60
    elif row_label == 'VervetMonkey': return 61
    elif row_label == 'Warthog': return 62
    elif row_label == 'Waterbuck': return 63
    elif row_label == 'WattledStarling': return 64
    elif row_label == 'WhiteHeadBuffaloWeaver': return 65
    elif row_label == 'Wildebeest': return 66
    elif row_label == 'WildebeestYouth': return 67
    elif row_label == 'Zebra': return 68
    elif row_label == 'ZebraYouth': return 69
    else : return 101 # Temporary bug fixed. Not good for model training
#     else: print('unlabelled!', row_label)

In [9]:
consensus_data['ASG0003lws']

[{'Babies': '0',
  'CaptureEventID': 'ASG0003lws',
  'Count': '2',
  'DateTime': '2011-05-15 00:27:10',
  'Eating': '0',
  'Evenness': '0',
  'Interacting': '0',
  'LocationX': '697344',
  'LocationY': '9731655',
  'Moving': '1',
  'NumBlanks': '1',
  'NumClassifications': '11',
  'NumImages': '1',
  'NumSpecies': '1',
  'NumVotes': '10',
  'Resting': '0.1',
  'SiteID': 'C03',
  'Species': 'wildebeest',
  'Standing': '0.3'}]

In [10]:
gold_standard_bb['ASG000c7bt.jpg']

[{'class': 'Wildebeest',
  'filename': 'ASG000c7bt.jpg',
  'height': '1536',
  'width': '2048',
  'xmax': '2048',
  'xmin': '1',
  'ymax': '1536',
  'ymin': '1'}]

In [11]:
all_images['ASG000yemj']

[{'CaptureEventID': 'ASG000yemj',
  'URL_Info': 'S6/C13/C13_R1/S6_C13_R1_IMAG0797.JPG'},
 {'CaptureEventID': 'ASG000yemj',
  'URL_Info': 'S6/C13/C13_R1/S6_C13_R1_IMAG0798.JPG'},
 {'CaptureEventID': 'ASG000yemj',
  'URL_Info': 'S6/C13/C13_R1/S6_C13_R1_IMAG0799.JPG'}]

In [23]:
#%%writefile data/data_model.py

def csvtodict(Project_filepath, bb_data, concensus_data, all_images_data, images):
    lst = []
    event_dict = {}
    csvfile = open(os.path.join(Project_filepath, bb_data), 'r')
    csvdata = csv.reader(csvfile, delimiter=',')
    first_row = next(csvdata)
    for row in csvdata:
        if row[0] in lst2 and row[0][0:10] not in event_dict: # the condition in lst2 is to pick only the images usd by schneider
            event_dict[row[0][0:10]] = {'metadata' : {"SiteID": consensus_data[row[0][0:10]][0]['SiteID'],
                                  "DateTime": consensus_data[row[0][0:10]][0]['DateTime'], 
                                  "Season": all_images['ASG000c7bt'][0]['URL_Info'][0:2]},
                                    'images' : [{"Path" : os.path.join(Project_filepath, row[0]),
                                "URL" : Project_filepath + all_images[row[0][0:10]][0]['URL_Info'],
                                "dim_x" : gold_standard_bb[row[0]][0]['width'],
                                "dim_y" : gold_standard_bb[row[0]][0]['height'],
                                "image_label" : "tbd", # This is the primary label in case we want to have some for the whole image
                                'observations' : [{'bb_ymin': v['ymin'], 
                                                   'bb_ymax': v['ymax'], 
                                                      'bb_primary_label': v['class'], 
                                                      'bb_xmin': v['xmin'], 
                                                      'bb_xmax': v['xmax'], 
                                                      'bb_label': {"species" : v['class'],
                                                    "pose" : "standing/ sitting/ running"
                                                }} for k, v in enumerate(gold_standard_bb[row[0]])]
                               }]
                                    }
    return event_dict


def dicttojson(event_dict):
    with open('event_dict.json', 'w') as outfile:
        json.dump(event_dict, outfile)
    #return json.dump(event_dict, outfile)

def jsontodict(event_json):
    with open(event_json, 'r') as f:
        return json.load(f)

""" creates tfrecord example from the dictionary element! """
def create_tf_example(data_dict):
    #with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = resize_jpeg((data_dict['images'][0]['Path']),  1000)
    #encoded_jpg_io = io.BytesIO(encoded_jpg)
    #image = Image.open(encoded_jpg_io)
    #width, height = image.size
    width = int(data_dict['images'][0]['dim_x'])
    height = int(data_dict['images'][0]['dim_y'])

    filename = data_dict['images'][0]['Path'].encode('utf-8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for bb_record in data_dict['images'][0]['observations']:
        xmins.append(float(bb_record['bb_xmin']) / width)
        xmaxs.append(float(bb_record['bb_xmax']) / width)
        ymins.append(float(bb_record['bb_ymin']) / height)
        ymaxs.append(float(bb_record['bb_ymax']) / height)
        classes_text.append(bb_record['bb_primary_label'].encode('utf8'))
        classes.append(class_text_to_int(bb_record['bb_primary_label']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    
    return tf_example

"""This iterates over each dictionary item, creates tf examples, 
    serializes the tfrecord examples and writes to a tfrecord file!!!"""
def encode_to_tfr_record(test_feature, out_tfr_file):
    with tf.python_io.TFRecordWriter(out_tfr_file) as writer:
        count = 0
        for k, v in test_feature.items():
            count+=1
            if count%500==0:
                print("processing event number %s : %s" % (count, k))
            example = create_tf_example(v)
            writer.write(example.SerializeToString())
            

Overwriting data/data_model.py


In [18]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [21]:
bb_data = '/home/rai00007/Desktop/zooniverse/Data/GoldStandardBoundBoxCoord.csv'
concensus_data = '/home/rai00007/Desktop/zooniverse/Data/consensus_data.csv'
all_images_data = '/home/rai00007/Desktop/zooniverse/Data/all_images.csv'
images = '/home/rai00007/Desktop/zooniverse/Data/'
event_dict = csvtodict(Project_filepath, bb_data, concensus_data, all_images_data, images) # In this step we create a dictionary from the csv
dicttojson(event_dict) # storing the dictionary to json
event_dict = jsontodict("event_dict.json") # getting back the dictionary. For creating the tfrecord we will use this 
# encode_to_tfr_record(event_dict, 'test_event.tfrecord') # this step creates a tfrecord file
event_dict['ASG000c7bt']

{'images': [{'Path': '/home/rai00007/Desktop/zooniverse/Data/images/ASG000c7bt.jpg',
   'URL': '/home/rai00007/Desktop/zooniverse/Data/images/S4/B07/B07_R2/S4_B07_R2_IMAG1488.JPG',
   'dim_x': '2048',
   'dim_y': '1536',
   'image_label': 'tbd',
   'observations': [{'bb_label': {'pose': 'standing/ sitting/ running',
      'species': 'Wildebeest'},
     'bb_primary_label': 'Wildebeest',
     'bb_xmax': '2048',
     'bb_xmin': '1',
     'bb_ymax': '1536',
     'bb_ymin': '1'}]}],
 'metadata': {'DateTime': '2012-02-22 11:01:38',
  'Season': 'S4',
  'SiteID': 'B07'}}

In [22]:
# see the tfrecord file
c = 0
for s_example in tf.python_io.tf_record_iterator('test_event.tfrecord'):
    example = tf.train.Example.FromString(s_example)
    c +=1
    if c == 1:
        print(example)
    else :
        break

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\002\356\003\350\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\

In [47]:
def decode_record(serialized_example):
    context_features = {
    #                     "image/filename" : tf.FixedLenFeature([], tf.string),
    #                     "image/format" : tf.FixedLenFeature([], tf.string),
                          "image/height" : tf.FixedLenFeature([], tf.int64),
    #                     "image/source_id" : tf.FixedLenFeature([], tf.string),
    #                     "image/width" : tf.FixedLenFeature([], tf.int64),
    #                     "image/encoded" :tf.FixedLenFeature([], tf.string)
                    }

    # sequence_features = {
    #                      "image/object/bbox/xmin" : tf.FixedLenSequenceFeature([], tf.int64),
    #                      "image/object/bbox/xmax" : tf.FixedLenSequenceFeature([], tf.int64),
    #                      "image/object/bbox/ymin" : tf.FixedLenSequenceFeature([], tf.int64),
    #                      "image/object/bbox/ymax" : tf.FixedLenSequenceFeature([], tf.int64)
    #                     }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
    #                                           sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return {}, {k: v for k, v in context.items()}

### Reading the TFRecord file

In [52]:
filename_list = ['test_event.tfrecord']
def create_tfr_reader(filename_list):
    batch_size = 2
    num_parallel_calls = 1
    drop_batch_remainder = False
    n_repeats = 1
    is_train = True
    buffer_size = 1

    dataset = tf.data.Dataset.from_tensor_slices(filename_list)

    dataset = dataset.apply(
            tf.contrib.data.parallel_interleave(
                lambda filename: tf.data.TFRecordDataset(filename),
                #sloppy=is_train,
                cycle_length=1)) #reads the filename as tf record
    dataset = dataset.prefetch(buffer_size=batch_size)

    # shuffle records only for training
    if is_train:
        dataset = dataset.apply(
                tf.contrib.data.shuffle_and_repeat(
                    buffer_size=buffer_size,
                    count=n_repeats))

    dataset = dataset.apply(
            tf.contrib.data.map_and_batch(
                  lambda x: decode_record(
                          serialized_example=x),
                          #output_labels=output_labels,
                          #label_lookup_dict=class_to_index_mappings,
                          #**kwargs),
                  batch_size=batch_size,
                  num_parallel_calls=num_parallel_calls,
                  drop_remainder=drop_batch_remainder))

    if not is_train:
        dataset = dataset.repeat(n_repeats)

    return dataset



In [55]:
# Example for inspecting TFR files
#import matplotlib.pyplot as plt

dataset = create_tfr_reader(filename_list)
iterator = dataset.make_one_shot_iterator()
batch_data = iterator.get_next()

#index_to_class = {0: 'wildebeest', 1: 'zebra'}

with tf.Session() as sess:
    for i in range(0, 1):
        features, labels = sess.run(batch_data)
        for j in range(0, labels['image/height'].shape[0]):
#             print("Class: %s" % index_to_class['class'][labels['label/class'][j][0]])
#             plt.imshow(features['image'][j, :, :, :])
#             plt.show()
            print labels['image/height'][j]

1536
1536


In [56]:
labels

{'image/height': array([1536, 1536])}

In [25]:
import inspect
lines = inspect.getsource(dataset_util.int64_feature)
print(lines)

#dataset_util.int64_feature

def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))



In [16]:
import glob
lst = (glob.glob("/home/rai00007/Desktop/zooniverse/Data/images/*.jpg"))
lst2 = []
for l in lst:
    lst2.append(l[-14:])

In [24]:
lst2 = []
for l in lst:
    lst2.append(l[-14:])

In [28]:
'ASGloldcz8.jpg' in lst2

False